# Creating the JSON data file in the relative directory

a sample intents.json file.

In [12]:
import json

# Data to be saved
data = {
    "intents": [
        {
            "tag": "greeting",
            "patterns": ["Hello", "Hi", "Hey", "Good morning", "Hi there"],
            "responses": ["Hello! How can I help?", "Hi there!", "Greetings!"]
        },
        {
            "tag": "goodbye",
            "patterns": ["Bye", "See you", "Goodbye", "Have a good day"],
            "responses": ["Goodbye!", "See you later!", "Have a great day!"]
        },
        {
            "tag": "thanks",
            "patterns": ["Thanks", "Thank you", "That helps"],
            "responses": ["You are welcome!", "Happy to help!", "No problem."]
        },
        {
            "tag": "name",
            "patterns": ["What is your name", "Who are you"],
            "responses": ["I am a chatbot created in Python.", "Call me Bot."]
        }
    ]
}

# Write to json file

with open('intents.json', 'w') as f:
    json.dump(data, f, indent=4)

print("intents.json created successfully!")

intents.json created successfully!



# Reading Indentations

Now we need to write the function to load this file. While the notebook mentions pandas, using the standard json library is much easier for nested data like this.

In [5]:
import json

def read_json(filename):
    with open(filename, 'r') as file:
        data = json.load(file) # we load the json format here 
    return data['intents']  # We return just the list of intents

# Call the function
intents_json = read_json('intents.json')

# Print to verify
print(f"Loaded {len(intents_json)} intents.")
print(intents_json[0]) # Print the first one to see structure

Loaded 4 intents.
{'tag': 'greeting', 'patterns': ['Hello', 'Hi', 'Hey', 'Good morning', 'Hi there'], 'responses': ['Hello! How can I help?', 'Hi there!', 'Greetings!']}


# Create Mappings from JSON pattern to Indents & Responses

This is the tricky part. We need two dictionaries:

- rgx2int: Maps a regex pattern to a tag (e.g., r"Hello|Hi|Hey" -> "greeting").

- int2res: Maps a tag to a list of responses.


In [8]:
def get_mappings(intents_list):
    rgx2int = {}
    int2res = {}
    
    for intent in intents_list:
        tag = intent['tag']
        
        # 1. Map Tag to Responses
        int2res[tag] = intent['responses']
        
        # 2. Map Patterns to Tag (Convert list of strings to single Regex)
        # We join the list ["Hello", "Hi"] into "Hello|Hi"
        pattern_string = "|".join(intent['patterns'])
        
        # Store in dictionary
        rgx2int[pattern_string] = tag # for every pattern add a tag
        
    return rgx2int, int2res

# Test it
rgx2int, int2res = get_mappings(intents_json)

print("Regex to Intent:", list(rgx2int.keys())[0], "->", list(rgx2int.values())[0])

Regex to Intent: Hello|Hi|Hey|Good morning|Hi there -> greeting


# Explaination

In [ ]:
# DONT RUN THIS PART IS ONLY TO EPXLAIN

    rgx2int[pattern_string] = tag

    rgx2int = {
        "Hello|Hi|Hey": "greeting"
    } # After this line runs (for the Greeting intent): You have added one entry.
    
    rgx2int = {
        "Hello|Hi|Hey": "greeting",
        "Bye|Goodbye": "goodbye"
    } # After the loop runs again (for the Goodbye intent): You add a second entry.


It will iterate through this dictionary and ask:

    "Does the user's input match Hello|Hi|Hey?"

        Yes? -> Then the meaningful tag is greeting.

    "Does the user's input match Bye|Goodbye?"

        Yes? -> Then the meaningful tag is goodbye.

# Example:

The .join() method takes a list of strings and "glues" them together using the character you specified (in this case, the pipe symbol |).

    Input (From JSON): ["Hello", "Hi", "Hey"]

    The Glue: "|"

    The Code: "|".join(["Hello", "Hi", "Hey"])

    The Result: "Hello|Hi|Hey"

In [6]:
# Imagine this came from your JSON file
patterns_list = ["Hello", "Hi", "Hey", "Good morning"]

# The magic line
pattern_string = "|".join(patterns_list)

print(f"Before: {type(patterns_list)} -> {patterns_list}")
print(f"After:  {type(pattern_string)} -> '{pattern_string}'")

# How the bot uses it:
import re
user_input = "Hi there chatbot"

if re.search(pattern_string, user_input):
    print("\nMatch found! The bot understood the greeting.")

Before: <class 'list'> -> ['Hello', 'Hi', 'Hey', 'Good morning']
After:  <class 'str'> -> 'Hello|Hi|Hey|Good morning'

Match found! The bot understood the greeting.


# Main loop Integeration

Here we combine everything into the final logic. We will look through the rgx2int dictionary, see if the user's input matches one of the regex keys, and if so, fetch the response.


In [11]:
import re
import random

# Load the data once globally (or inside the function as per your notebook template)
intents_list = read_json('intents.json')
rgx2int, int2res = get_mappings(intents_list)

def generate_response(user_input):
    # Check every pattern in our dictionary
    for pattern, tag in rgx2int.items():
        # Using re.search to find the pattern in the input
        # re.IGNORECASE makes "hello" match "Hello"
        if re.search(pattern, user_input, re.IGNORECASE):
            # If match found, get the list of responses for this tag
            possible_responses = int2res[tag]
            # Return a random response from that list
            return random.choice(possible_responses)
            
    return False # Return False if no match found

def chatbot():
    print("Chatbot: Welcome! (Type 'bye' to exit)")
    
    while True:
        user_input = input("You: ")
        
        # Exit condition
        if user_input.lower() == "bye":
            print("Chatbot: Goodbye!")
            break
            
        # Generate response
        response = generate_response(user_input)
        
        if response:
            print(f"Chatbot: {response}")
        else:
            print("Chatbot: I'm sorry, I don't understand that.")

# Run it
chatbot()

Chatbot: Welcome! (Type 'bye' to exit)


You:  Hello My name is Marawan


Chatbot: Hello! How can I help?


You:  What is your name ?


Chatbot: I am a chatbot created in Python.


...

Chatbot: Call me Bot.


You:  Goodbye


Chatbot: Have a great day!


You:  bye


Chatbot: Goodbye!
